# Function decorators and closures

Function decorators let us “mark” functions in the source code to enhance their behavior is some way. This is powerful stuff, but mastering it requires understanding closures.

Aside from their application in decorators, closures are also essential for effective asyn‐ chronous programming with callbacks, and for coding in a functional style whenever it makes sense.

## Decorators

A decorator is a callable that takes another function as argument. Strictly speaking, decorators are just syntactic sugar. The second crucial fact is that they are executed immediately when a module is loaded.

In [39]:
#@decorate
#def target():
#    print('running target()')
    
#Has the same effect as writing this:

#def target():
#    print('running target()')
    
#target = decorate(target)

#live example

def deco(func):
    def inner():
        print('running inner()')
    #deco just returns inner()
    return inner

#deco used as a decorator
@deco
def target():
    print('running target()')

#instantiating target points to inner()
print(target())
#target is now therefore a reference to inner
print(target)

running inner()
None
<function deco.<locals>.inner at 0x11174f620>


In [ ]:
## Closure example - How to create a closure?

-  create a nested function
-  nested function has to refer to a variable defined inside the enclosing function
-  enclosing function has to return the nested function

In [40]:
def makeInc(x):
    
    def inc(y):
        # x is "closed" in the definition of inc
        print("y = %d" %y)
        return y + x

    print("x = %d" %x)
    return inc

#instantiate makeInc, with x as 5
inc5 = makeInc(5)
print(inc5(5)) # returns 10

#instantiate makeInc, with x as 10
inc10 = makeInc(10)
print(inc10(5)) # returns 15

x = 5
y = 5
10
x = 10
y = 5
15


In [45]:
#registry will hold references to functions decorated by @register
registry = []

#register takes a function as argument.
def register(func):
    #Display what function is being decorated, for demonstration
    print('running register function (%s)' % func)
    #Append func to registry list
    registry.append(func)
    #Return func: we must return a function, here we return the same received as argument.
    return func

#f1 and f2 are decorated by @register
@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

#f3 is not decorated.
def f3():
    print('running f3()')

#main displays the registry, then calls f1(), f2() and f3().
def main():
    print('running main()') 
    print('registry ->', registry) 
    f1()
    f2()
    f3()
    
if __name__=='__main__': main()
    
#If registration.py is imported (and not run as a script), the output is this:
#>>> import registration
#running register(<function f1 at 0x10063b1e0>) 
#running register(<function f2 at 0x10063b268>)

running register function (<function f1 at 0x111757f28>)
running register function (<function f2 at 0x1117579d8>)
running main()
registry -> [<function f1 at 0x111757f28>, <function f2 at 0x1117579d8>]
running f1()
running f2()
running f3()


## Decorator-enhanced Strategy pattern



In [53]:
promos = []


def promotion(promo_func):
    promos.append(promo_func)
    return promo_func


@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units""" 
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07 
    return 0


def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

b=6
def f2(a):
    print(a)
    print(b)
    #if b is deleted, the function will fetch it from the global scope
#     b=9

# when Python compiles the body of the function, it decides that b is a local variable 
# because it is assigned within the function. 
# The generated bytecode reflects this decision and will try to fetch b from the local environment. 
# aka UnboundLocalError: local variable 'b' referenced before assignment
f2(3)


3
6


## Closures

A closure is function with an extended scope that encompasses non-global variables referenced in the body of the function but not defined there. It does not matter whether the function is anonymous or not, what matters is that it can access non-global variables that are defined outside of its body.

In [71]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value): 
        self.series.append(new_value) 
        total = sum(self.series) 
        return total/len(self.series)

avg = Averager()
print(avg(10))
print(avg(50))
print(avg(40))

def make_averager():
    series = []
    
    def averager(new_value):
        #Within averager, series is a free variable. 
        series.append(new_value) 
        total = sum(series) 
        return total/len(series)
    
    return averager

avg2 = make_averager()
print(avg2(10))
print(avg2(50))
print(avg2(40))
print(avg2.__code__.co_varnames)
print(avg2.__code__.co_freevars)

def make_averager2(): 
    count = 0
    total = 0
    
    def averager(new_value):
        #count is being called, but it is only in the local scope, so has not yet been assigned
        #if nonlocal is used, Python knows to look outside the local scope
        nonlocal count, total
        count += 1
        total += new_value 
        return total / count
    
    return averager

avg3 = make_averager2()
#throws UnboundLocalError: local variable 'count' referenced before assignment, when nonlocal is missing
print(avg3(10))

10.0
30.0
33.333333333333336
10.0
30.0
33.333333333333336
('new_value', 'total')
('series',)
10.0


In [77]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs) 
        elapsed = time.time() - t0 
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args)) 
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result)) 
        return result
    return clocked

@clock
def snooze(seconds): time.sleep(seconds)

#From now on, each time factorial(n) is called, clocked(n) gets executed
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)') 
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)') 
    print('6! =', factorial(6))


**************************************** Calling snooze(.123)
[0.12658882s] snooze(0.123) -> None 
**************************************** Calling factorial(6)
[0.00000119s] factorial(1) -> 1 
[0.00015378s] factorial(2) -> 2 
[0.00022388s] factorial(3) -> 6 
[0.00028610s] factorial(4) -> 24 
[0.00036502s] factorial(5) -> 120 
[0.00059915s] factorial(6) -> 720 
6! = 720


## Decorators in the standard library

Python has three built-in functions that are designed to decorate methods: property, classmethod and staticmethod. 

### Memoization with functools.lru_cache

A very practical decorator is functools.lru_cache. It implements memoization: an optimization technique which works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments. The letters LRU stand for Least Recently Used, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while.

In [82]:
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs) 
        elapsed = time.time() - t0 
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args)) 
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result)) 
        return result
    return clocked

@clock
def fibonacci(n):
    #fibonacci(1) is called 8 times, fibonacci(2) 5 times
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__': 
    print(fibonacci(6))
    
import functools


@functools.lru_cache() # 
@clock #
def fibonacci2(n):
    if n < 2: 
        return n
    return fibonacci2(n-2) + fibonacci2(n-1) 

if __name__=='__main__':
    print(fibonacci2(6))

[0.00000119s] fibonacci(0) -> 0 
[0.00000095s] fibonacci(1) -> 1 
[0.00046587s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(1) -> 1 
[0.00000072s] fibonacci(0) -> 0 
[0.00000119s] fibonacci(1) -> 1 
[0.00008774s] fibonacci(2) -> 1 
[0.00020289s] fibonacci(3) -> 2 
[0.00088596s] fibonacci(4) -> 3 
[0.00000095s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000119s] fibonacci(1) -> 1 
[0.00016093s] fibonacci(2) -> 1 
[0.00043392s] fibonacci(3) -> 2 
[0.00000000s] fibonacci(0) -> 0 
[0.00000095s] fibonacci(1) -> 1 
[0.00006676s] fibonacci(2) -> 1 
[0.00000119s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00006104s] fibonacci(2) -> 1 
[0.00012112s] fibonacci(3) -> 2 
[0.00024986s] fibonacci(4) -> 3 
[0.00076199s] fibonacci(5) -> 5 
[0.00172329s] fibonacci(6) -> 8 
8
[0.00000000s] fibonacci2(0) -> 0 
[0.00000095s] fibonacci2(1) -> 1 
[0.00027204s] fibonacci2(2) -> 1 
[0.00000215s] fibonacci2(3) -> 2 
[0.00058532s] fibonacci2(4) -> 3 
[0.

## Generic functions with single dispatch

The new functools.singledispatch decorator in Python 3.4 allows each module to contribute to the overall solution, and lets you easily provide a specialized function even for classes that you can’t edit. If you decorate a plain function with @singledispatch it becomes a generic function: a group of functions to perform the same operation in different ways, depending on the type of the first argument.

A notable quality of the singledispatch mechanism is that you can register specialized functions anywhere in the system, in any module. If you later add a module with a new user-defined type, you can easily provide a new custom function to handle that type. And you can write custom functions for classes that you did not write and can’t change.

In [87]:
from functools import singledispatch 
from collections import abc
import numbers
import html

#@singledispatch marks the base function which handles the object type.
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj)) 
    return '<pre>{}</pre>'.format(content)

#Each specialized function is decorated with @«base_function».register(«type»).
@htmlize.register(str)
#The name of the specialized functions is irrelevant; _ is a good choice to make this clear.
def _(text):
    content = html.escape(text).replace('\n', '<br>\n') 
    return '<p>{0}</p>'.format(content)

#For each additional type to receive special treatment, register a new function.
#numbers.Integral is a virtual superclass of int (see below).
@htmlize.register(numbers.Integral) 
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

#You can stack several register decorators to support different types with the
#same function.
@htmlize.register(tuple) 
@htmlize.register(abc.MutableSequence) 
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq) 
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## Parametrized Decorators

When parsing a decorator in source code, Python takes the decorated function and passes it as the first argument to the decorator function.

In [94]:
#registry is now a set, so adding and removing functions is faster.
registry = set()

#register takes an optional keyword argument.
def register(active=True):
        #The decorate inner function is the actual decorator; note how it takes a function as argument.
        def decorate(func):
            print('running register(active=%s)->decorate(%s)' % (active, func))
            #Register func only if the active argument (retrieved from the closure) is True.
            if active:
                print("TRUE")
                registry.add(func)
            else:
                print("FALSE")
                #If not active and func in registry, remove it.
                registry.discard(func)
            
            #Because decorate is a decorator, it must return a function.
            return func
        #register is our decorator factory, so it returns decorate.
        return decorate
    
@register(active=False) 
def f1():
    print('running f1()')
    
@register() 
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

#only f2 is added to the registry
print(registry)
#f3 is added
register()(f3)
print(registry)
#f2 is removed
register(active=False)(f2)
print(registry)

running register(active=False)->decorate(<function f1 at 0x111736840>)
FALSE
running register(active=True)->decorate(<function f2 at 0x111757840>)
TRUE
{<function f2 at 0x111757840>}
running register(active=True)->decorate(<function f3 at 0x111757158>)
TRUE
{<function f2 at 0x111757840>, <function f3 at 0x111757158>}
running register(active=False)->decorate(<function f2 at 0x111757840>)
FALSE
{<function f3 at 0x111757158>}


In [97]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

#clock is our parametrized decorator factory.
def clock(fmt=DEFAULT_FMT):
    #decorate is the actual decorator.
    def decorate(func):
        #clocked wraps the decorated function.
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args) 
            elapsed = time.time() - t0 
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) 
            result = repr(_result) 
            print(fmt.format(**locals()))
            return _result
        return clocked 
    return decorate
    
if __name__ == '__main__':
    
    @clock()
    def snooze(seconds):
        time.sleep(seconds) 
    
    for i in range(3):
            snooze(.123)

[0.12328005s] snooze(0.123) -> None
[0.12703919s] snooze(0.123) -> None
[0.12393713s] snooze(0.123) -> None
